# Realtime ISL

## Using MediaPipe and Classification algorithm

In [ ]:
import cv2
import mediapipe as mp 
import joblib
import numpy as np 

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# For webcam input:
hands = mp_hands.Hands(
    min_detection_confidence=0.7, min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

def data_clean(landmark):
  
  data = landmark[0]
  
  try:
    data = str(data)

    data = data.strip().split('\n')

    garbage = ['landmark {', '  visibility: 0.0', '  presence: 0.0', '}']

    without_garbage = []

    for i in data:
        if i not in garbage:
            without_garbage.append(i)

    clean = []

    for i in without_garbage:
        i = i.strip()
        clean.append(i[2:])

    for i in range(0, len(clean)):
        clean[i] = float(clean[i])

    
    return([clean])

  except:
    return(np.zeros([1,63], dtype=int)[0])

while cap.isOpened():
  success, image = cap.read()
  
  image = cv2.flip(image, 1)
  
  if not success:
    break

  # Flip the image horizontally for a later selfie-view display, and convert
  # the BGR image to RGB.
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  # To improve performance, optionally mark the image as not writeable to
  # pass by reference.
  image.flags.writeable = False
  results = hands.process(image)

  # Draw the hand annotations on the image.
  image.flags.writeable = True

  image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

  if results.multi_hand_landmarks:
    for hand_landmarks in results.multi_hand_landmarks:
      mp_drawing.draw_landmarks(
          image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    cleaned_landmark = data_clean(results.multi_hand_landmarks)
    #print(cleaned_landmark)

    if cleaned_landmark:
      clf = joblib.load('model.pkl')
      y_pred = clf.predict(cleaned_landmark)
      image = cv2.putText(image, str(y_pred[0]), (50,150), cv2.FONT_HERSHEY_SIMPLEX,  3, (0,0,255), 2, cv2.LINE_AA) 
    
  cv2.imshow('MediaPipe Hands', image)
  
  if cv2.waitKey(5) & 0xFF == 27:
    break

hands.close()
cap.release()
cv2.destroyAllWindows()

In [ ]:

        """
		st.info("CLASSIFICATION REPORT")
		cr = Image.open('Results/classification_report.png')
		st.image(cr, width=320)

		st.info("CONFUSION MATRIX")
		cm = Image.open('Results/confusion_matrix.png')
		st.image(cm, width=320)

		st.info("ACCURACY SCORE")
		acs = Image.open('Results/accuracy_score.png')
		st.image(acs, width=320)
        """

In [28]:
from PIL import Image, ImageTk
import tkinter as tk
import cv2
import os
import numpy as np
import operator
import time
import sys, os
import matplotlib.pyplot as plt
from string import ascii_uppercase
import mediapipe as mp 
import joblib
import numpy as np
class Application:
    def __init__(self):
        self.directory = 'BEP3'
        print("Loaded model from disk")
        self.root = tk.Tk()
        self.root.title("Indian Sign Language Alphabet recognition")
        self.root.protocol('WM_DELETE_WINDOW', self.destructor)
        self.root.geometry("800x600")
        self.panel = tk.Label(self.root)
        self.panel.place(x = 135, y = 10, width = 640, height = 640)
        self.panel2 = tk.Label(self.root) # initialize image panel
        self.panel2.place(x = 460, y = 95, width = 310, height = 310)
        
        self.T = tk.Label(self.root)
        self.T.place(x=31,y = 17)
        self.T.config(text = "Indian Sign Language A-Z",font=("Helvetica",30,"bold"))
        self.panel3 = tk.Label(self.root) # Current SYmbol
        self.panel3.place(x = 50,y=64)
        self.T1 = tk.Label(self.root)
        self.T1.place(x = 10,y = 64)
        self.T1.config(text="Character :",font=("Helvetica",40,"bold"))
        self.panel4 = tk.Label(self.root) # Word
        self.panel4.place(x = 22,y=70)
        self.T2 = tk.Label(self.root)
        self.T2.place(x = 10,y = 70)
        self.T2.config(text ="Word :",font=("Helvetica",40,"bold"))
        self.panel5 = tk.Label(self.root) # Sentence
        self.panel5.place(x = 35,y=76)
        self.T3 = tk.Label(self.root)
        self.T3.place(x = 10,y = 76)
        self.T3.config(text ="Sentence :",font=("Helvetica",40,"bold"))

        self.btcall = tk.Button(self.root,command = self.action_call,height = 0,width = 0)
        self.btcall.config(text = "About",font = ("Arial",14))
        self.btcall.place(x = 700, y = 0)

        self.bt1=tk.Button(self.root, command=self.action2,height = 0,width = 0)
        self.bt1.place(x = 26,y=89)
        #self.bt1.grid(padx = 10, pady = 10)
        self.bt2=tk.Button(self.root, command=self.action2,height = 0,width = 0)
        self.bt2.place(x = 32,y=89)
        #self.panel3.place(x = 10,y=660)
        # self.bt2.grid(row = 4, column = 1, columnspan = 1, padx = 10, pady = 10, sticky = tk.NW)
        self.bt3=tk.Button(self.root, command=self.action2,height = 0,width = 0)
        self.bt3.place(x = 65,y=89)
        # self.bt3.grid(row = 4, column = 2, columnspan = 1, padx = 10, pady = 10, sticky = tk.NW)
        self.bt4=tk.Button(self.root, command=self.action2,height = 0,width = 0)
        self.bt4.place(x = 15,y=95)
        # self.bt4.grid(row = bt1, column = 0, columnspan = 1, padx = 10, pady = 10, sticky = tk.N)
        self.bt5=tk.Button(self.root, command=self.action2,height = 0,width = 0)
        self.bt5.place(x = 42,y=95)
        # self.bt5.grid(row = 5, column = 1, columnspan = 1, padx = 10, pady = 10, sticky = tk.N)
        self.str=""
        self.word=""
        self.current_symbol="Empty"
        self.photo="Empty"
    
    def destructor(self):
        print("Closing Application...")
        self.root.destroy()
        self.vs.release()
        cv2.destroyAllWindows()
        
    def action2(self):
        predicts=self.hs.suggest(self.word)
        if(len(predicts) > 1):
            self.word=""
            self.str+=" "
            self.str+=predicts[1]
    
    def destructor1(self):
        print("Closing Application...")
        self.root1.destroy()
        
        
    def action_call(self) :
        
        self.root1 = tk.Toplevel(self.root)
        self.root1.title("About")
        self.root1.protocol('WM_DELETE_WINDOW', self.destructor1)
        self.root1.geometry("800x600")
        
        self.tx = tk.Label(self.root1)
        self.tx.place(x = 300,y = 20)
        self.tx.config(text = "Efforts By", fg="red", font = ("Helvetica",30,"bold"))

        
        #self.photo3 = tk.PhotoImage(file='Pictures/luv.png')
        #self.w3 = tk.Label(self.root1, image = self.photo3)
        #self.w3.place(x = 38, y = 10)
        self.tx3 = tk.Label(self.root1)
        self.tx3.place(x = 300,y = 80)
        self.tx3.config(text = "Ayushman Rayaguru", font = ("Helvetica",15,"bold"))

        #self.photo4 = tk.PhotoImage(file='Pictures/sheldon.png')
        #self.w4 = tk.Label(self.root1, image = self.photo4)
        #self.w4.place(x = 56, y = 15)
        self.tx4 = tk.Label(self.root1)
        self.tx4.place(x = 300,y = 130)
        self.tx4.config(text = "Sampreeti Saha", font = ("Helvetica",15,"bold"))
        
        #self.photo5 = tk.PhotoImage(file='Pictures/sid.png')
        #self.w5 = tk.Label(self.root1, image = self.photo5)
        #self.w5.place(x = 74, y = 15)
        self.tx5 = tk.Label(self.root1)
        self.tx5.place(x = 300,y = 180)
        self.tx5.config(text = "Kiransingh Pal", font = ("Helvetica",15,"bold"))
        
        self.tx7 = tk.Label(self.root1)
        self.tx7.place(x = 300,y = 350)
        self.tx7.config(text = "Under Guidance of", fg="red", font = ("Helvetica",30,"bold"))

        #self.photo6 = tk.PhotoImage(file='Pictures/sir.png')
        #self.w6 = tk.Label(self.root1, image = self.photo6)
       # self.w6.place(x = 35, y = 42)
        self.tx6 = tk.Label(self.root1)
        self.tx6.place(x = 10,y = 400)
        self.tx6.config(text = "Dr.Mausami Turuk", fg="red",font = ("Helvetica",30,"bold"))
print("Starting Application...")
pba = Application()
pba.root.mainloop()



Starting Application...
Loaded model from disk
Closing Application...
Closing Application...
Closing Application...
Closing Application...
Closing Application...
Closing Application...
Closing Application...
Closing Application...
Closing Application...


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Ayushman\AppData\Local\Programs\Python\Python37\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "C:\Users\Ayushman\AppData\Local\Temp/ipykernel_2352/2976149570.py", line 78, in destructor
    self.vs.release()
AttributeError: 'Application' object has no attribute 'vs'


In [ ]:
from PIL import Image, ImageTk
import tkinter as tk
import cv2
import os
import numpy as np
import operator
import time
import sys, os
import matplotlib.pyplot as plt
from string import ascii_uppercase
import mediapipe as mp 
import joblib
import numpy as np 

class Application:
    def __init__(self):
	self.directory = 'model'
        self.vs = cv2.VideoCapture(0)
        self.current_image = None
        self.current_image2 = None
        
        self.json_file = open(self.directory+"model-bw.json", "r")
        self.model_json = self.json_file.read()
        self.json_file.close()
        self.loaded_model = model_from_json(self.model_json)
        self.loaded_model.load_weights(self.directory+"model-bw.h5")

        self.json_file_dru = open(self.directory+"model-bw_dru.json" , "r")
        self.model_json_dru = self.json_file_dru.read()
        self.json_file_dru.close()
        self.loaded_model_dru = model_from_json(self.model_json_dru)
        self.loaded_model_dru.load_weights("model-bw_dru.h5")

        self.json_file_tkdi = open(self.directory+"model-bw_tkdi.json" , "r")
        self.model_json_tkdi = self.json_file_tkdi.read()
        self.json_file_tkdi.close()
        self.loaded_model_tkdi = model_from_json(self.model_json_tkdi)
        self.loaded_model_tkdi.load_weights(self.directory+"model-bw_tkdi.h5")

        self.json_file_smn = open(self.directory+"model-bw_smn.json" , "r")
        self.model_json_smn = self.json_file_smn.read()
        self.json_file_smn.close()
        self.loaded_model_smn = model_from_json(self.model_json_smn)
        self.loaded_model_smn.load_weights(self.directory+"model-bw_smn.h5")
        
        self.ct = {}
        self.ct['blank'] = 0
        self.blank_flag = 0
        for i in ascii_uppercase:
          self.ct[i] = 0
        print("Loaded model from disk")
        self.root = tk.Tk()
        self.root.title("Sign language to Text Converter")
        self.root.protocol('WM_DELETE_WINDOW', self.destructor)
        self.root.geometry("900x1100")
        self.panel = tk.Label(self.root)
        self.panel.place(x = 135, y = 10, width = 640, height = 640)
        self.panel2 = tk.Label(self.root) # initialize image panel
        self.panel2.place(x = 460, y = 95, width = 310, height = 310)
        
        self.T = tk.Label(self.root)
        self.T.place(x=31,y = 17)
        self.T.config(text = "Sign Language to Text",font=("courier",40,"bold"))
        self.panel3 = tk.Label(self.root) # Current SYmbol
        self.panel3.place(x = 500,y=640)
        self.T1 = tk.Label(self.root)
        self.T1.place(x = 10,y = 640)
        self.T1.config(text="Character :",font=("Courier",40,"bold"))
        self.panel4 = tk.Label(self.root) # Word
        self.panel4.place(x = 220,y=700)
        self.T2 = tk.Label(self.root)
        self.T2.place(x = 10,y = 700)
        self.T2.config(text ="Word :",font=("Courier",40,"bold"))
        self.panel5 = tk.Label(self.root) # Sentence
        self.panel5.place(x = 350,y=760)
        self.T3 = tk.Label(self.root)
        self.T3.place(x = 10,y = 760)
        self.T3.config(text ="Sentence :",font=("Courier",40,"bold"))

        self.T4 = tk.Label(self.root)
        self.T4.place(x = 250,y = 820)
        self.T4.config(text = "Suggestions",fg="red",font = ("Courier",40,"bold"))

        self.btcall = tk.Button(self.root,command = self.action_call,height = 0,width = 0)
        self.btcall.config(text = "About",font = ("Courier",14))
        self.btcall.place(x = 825, y = 0)

        self.bt1=tk.Button(self.root, command=self.action1,height = 0,width = 0)
        self.bt1.place(x = 26,y=890)
        #self.bt1.grid(padx = 10, pady = 10)
        self.bt2=tk.Button(self.root, command=self.action2,height = 0,width = 0)
        self.bt2.place(x = 325,y=890)
        #self.panel3.place(x = 10,y=660)
        # self.bt2.grid(row = 4, column = 1, columnspan = 1, padx = 10, pady = 10, sticky = tk.NW)
        self.bt3=tk.Button(self.root, command=self.action3,height = 0,width = 0)
        self.bt3.place(x = 625,y=890)
        # self.bt3.grid(row = 4, column = 2, columnspan = 1, padx = 10, pady = 10, sticky = tk.NW)
        self.bt4=tk.Button(self.root, command=self.action4,height = 0,width = 0)
        self.bt4.place(x = 125,y=950)
        # self.bt4.grid(row = bt1, column = 0, columnspan = 1, padx = 10, pady = 10, sticky = tk.N)
        self.bt5=tk.Button(self.root, command=self.action5,height = 0,width = 0)
        self.bt5.place(x = 425,y=950)
        # self.bt5.grid(row = 5, column = 1, columnspan = 1, padx = 10, pady = 10, sticky = tk.N)
        self.str=""
        self.word=""
        self.current_symbol="Empty"
        self.photo="Empty"
        self.video_loop()

    def video_loop(self):
        ok, frame = self.vs.read()
        if ok:
            cv2image = cv2.flip(frame, 1)
            x1 = int(0.5*frame.shape[1])
            y1 = 10
            x2 = frame.shape[1]-10
            y2 = int(0.5*frame.shape[1])
            cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255,0,0) ,1)
            cv2image = cv2.cvtColor(cv2image, cv2.COLOR_BGR2RGBA)
            self.current_image = Image.fromarray(cv2image)
            imgtk = ImageTk.PhotoImage(image=self.current_image)
            self.panel.imgtk = imgtk
            self.panel.config(image=imgtk)
            cv2image = cv2image[y1:y2, x1:x2]
            gray = cv2.cvtColor(cv2image, cv2.COLOR_BGR2GRAY)
            blur = cv2.GaussianBlur(gray,(5,5),2)
            th3 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
            ret, res = cv2.threshold(th3, 70, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
            self.predict(res)
            self.current_image2 = Image.fromarray(res)
            imgtk = ImageTk.PhotoImage(image=self.current_image2)
            self.panel2.imgtk = imgtk
            self.panel2.config(image=imgtk)
            self.panel3.config(text=self.current_symbol,font=("Courier",50))
            self.panel4.config(text=self.word,font=("Courier",40))
            self.panel5.config(text=self.str,font=("Courier",40))
            predicts=self.hs.suggest(self.word)
            if(len(predicts) > 0):
                self.bt1.config(text=predicts[0],font = ("Courier",20))
            else:
                self.bt1.config(text="")
            if(len(predicts) > 1):
                self.bt2.config(text=predicts[1],font = ("Courier",20))
            else:
                self.bt2.config(text="")
            if(len(predicts) > 2):
                self.bt3.config(text=predicts[2],font = ("Courier",20))
            else:
                self.bt3.config(text="")
            if(len(predicts) > 3):
                self.bt4.config(text=predicts[3],font = ("Courier",20))
            else:
                self.bt4.config(text="")
            if(len(predicts) > 4):
                self.bt4.config(text=predicts[4],font = ("Courier",20))
            else:
                self.bt4.config(text="")                
        self.root.after(30, self.video_loop)
    def predict(self,test_image):
        test_image = cv2.resize(test_image, (128,128))
        result = self.loaded_model.predict(test_image.reshape(1, 128, 128, 1))
        result_dru = self.loaded_model_dru.predict(test_image.reshape(1 , 128 , 128 , 1))
        result_tkdi = self.loaded_model_tkdi.predict(test_image.reshape(1 , 128 , 128 , 1))
        result_smn = self.loaded_model_smn.predict(test_image.reshape(1 , 128 , 128 , 1))
        prediction={}
        prediction['blank'] = result[0][0]
        inde = 1
        for i in ascii_uppercase:
            prediction[i] = result[0][inde]
            inde += 1
        #LAYER 1
        prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)
        self.current_symbol = prediction[0][0]
        #LAYER 2
        if(self.current_symbol == 'D' or self.current_symbol == 'R' or self.current_symbol == 'U'):
        	prediction = {}
        	prediction['D'] = result_dru[0][0]
        	prediction['R'] = result_dru[0][1]
        	prediction['U'] = result_dru[0][2]
        	prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)
        	self.current_symbol = prediction[0][0]

        if(self.current_symbol == 'D' or self.current_symbol == 'I' or self.current_symbol == 'K' or self.current_symbol == 'T'):
        	prediction = {}
        	prediction['D'] = result_tkdi[0][0]
        	prediction['I'] = result_tkdi[0][1]
        	prediction['K'] = result_tkdi[0][2]
        	prediction['T'] = result_tkdi[0][3]
        	prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)
        	self.current_symbol = prediction[0][0]

        if(self.current_symbol == 'M' or self.current_symbol == 'N' or self.current_symbol == 'S'):
        	prediction1 = {}
        	prediction1['M'] = result_smn[0][0]
        	prediction1['N'] = result_smn[0][1]
        	prediction1['S'] = result_smn[0][2]
        	prediction1 = sorted(prediction1.items(), key=operator.itemgetter(1), reverse=True)
        	if(prediction1[0][0] == 'S'):
        		self.current_symbol = prediction1[0][0]
        	else:
        		self.current_symbol = prediction[0][0]
        if(self.current_symbol == 'blank'):
            for i in ascii_uppercase:
                self.ct[i] = 0
        self.ct[self.current_symbol] += 1
        if(self.ct[self.current_symbol] > 60):
            for i in ascii_uppercase:
                if i == self.current_symbol:
                    continue
                tmp = self.ct[self.current_symbol] - self.ct[i]
                if tmp < 0:
                    tmp *= -1
                if tmp <= 20:
                    self.ct['blank'] = 0
                    for i in ascii_uppercase:
                        self.ct[i] = 0
                    return
            self.ct['blank'] = 0
            for i in ascii_uppercase:
                self.ct[i] = 0
            if self.current_symbol == 'blank':
                if self.blank_flag == 0:
                    self.blank_flag = 1
                    if len(self.str) > 0:
                        self.str += " "
                    self.str += self.word
                    self.word = ""
            else:
                if(len(self.str) > 16):
                    self.str = ""
                self.blank_flag = 0
                self.word += self.current_symbol
    def action1(self):
    	predicts=self.hs.suggest(self.word)
    	if(len(predicts) > 0):
            self.word=""
            self.str+=" "
            self.str+=predicts[0]
    def action2(self):
    	predicts=self.hs.suggest(self.word)
    	if(len(predicts) > 1):
            self.word=""
            self.str+=" "
            self.str+=predicts[1]
    def action3(self):
    	predicts=self.hs.suggest(self.word)
    	if(len(predicts) > 2):
            self.word=""
            self.str+=" "
            self.str+=predicts[2]
    def action4(self):
    	predicts=self.hs.suggest(self.word)
    	if(len(predicts) > 3):
            self.word=""
            self.str+=" "
            self.str+=predicts[3]
    def action5(self):
    	predicts=self.hs.suggest(self.word)
    	if(len(predicts) > 4):
            self.word=""
            self.str+=" "
            self.str+=predicts[4]
    def destructor(self):
        print("Closing Application...")
        self.root.destroy()
        self.vs.release()
        cv2.destroyAllWindows()
    
    def destructor1(self):
        print("Closing Application...")
        self.root1.destroy()

    def action_call(self) :
        
        self.root1 = tk.Toplevel(self.root)
        self.root1.title("About")
        self.root1.protocol('WM_DELETE_WINDOW', self.destructor1)
        self.root1.geometry("900x900")
        
        # img = cv2.imread("Pictures/sir.jpg", 1)
        # # img = cv2.resize(img, (300, 300))
        # cv2.imwrite("Pictures/sir.png", img)
        # return 
        
        self.tx = tk.Label(self.root1)
        self.tx.place(x = 330,y = 20)
        self.tx.config(text = "Efforts By", fg="red", font = ("Courier",30,"bold"))

        self.photo1 = tk.PhotoImage(file='Pictures/ravi.png')
        self.w1 = tk.Label(self.root1, image = self.photo1)
        self.w1.place(x = 20, y = 105)
        self.tx6 = tk.Label(self.root1)
        self.tx6.place(x = 20,y = 250)
        self.tx6.config(text = "RC\nIIT2016141", font = ("Courier",15,"bold"))

        self.photo2 = tk.PhotoImage(file='Pictures/nitin.png')
        self.w2 = tk.Label(self.root1, image = self.photo2)
        self.w2.place(x = 200, y = 105)
        self.tx2 = tk.Label(self.root1)
        self.tx2.place(x = 200,y = 250)
        self.tx2.config(text = "Nitin\nIIT2016132", font = ("Courier",15,"bold"))

        
        self.photo3 = tk.PhotoImage(file='Pictures/luv.png')
        self.w3 = tk.Label(self.root1, image = self.photo3)
        self.w3.place(x = 380, y = 105)
        self.tx3 = tk.Label(self.root1)
        self.tx3.place(x = 380,y = 250)
        self.tx3.config(text = "Luv\nIIT2016085", font = ("Courier",15,"bold"))

        self.photo4 = tk.PhotoImage(file='Pictures/sheldon.png')
        self.w4 = tk.Label(self.root1, image = self.photo4)
        self.w4.place(x = 560, y = 105)
        self.tx4 = tk.Label(self.root1)
        self.tx4.place(x = 560,y = 250)
        self.tx4.config(text = "Sheldon\nIIT2016137", font = ("Courier",15,"bold"))
        
        self.photo5 = tk.PhotoImage(file='Pictures/sid.png')
        self.w5 = tk.Label(self.root1, image = self.photo5)
        self.w5.place(x = 740, y = 105)
        self.tx5 = tk.Label(self.root1)
        self.tx5.place(x = 740,y = 250)
        self.tx5.config(text = "Siddhant\nIIT2016069", font = ("Courier",15,"bold"))
        
        self.tx7 = tk.Label(self.root1)
        self.tx7.place(x = 170,y = 360)
        self.tx7.config(text = "Under the supervision of", fg="red", font = ("Courier",30,"bold"))

        self.photo6 = tk.PhotoImage(file='Pictures/sir.png')
        self.w6 = tk.Label(self.root1, image = self.photo6)
        self.w6.place(x = 350, y = 420)
        self.tx6 = tk.Label(self.root1)
        self.tx6.place(x = 230,y = 670)
        self.tx6.config(text = "Dr. Vrijendra Singh", font = ("Courier",30,"bold"))

print("Starting Application...")
pba = Application()
pba.root.mainloop()

In [1]:
pip install streamlit

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=e7f31181b7ce11fee2678801fc322886f5e61d0606eaa9bba5a420ff4eed84a4
  Stored in directory: c:\users\ayushman\appdata\local\pip\cache\wheels\22\f5\18\df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Ayushman\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


In [3]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
from PIL import Image,ImageEnhance
import matplotlib.pyplot as plt
import streamlit as st
import pandas as pd
import numpy as np
import argparse
import imutils
import time
import glob
import random
import cv2
import os


st.set_option('deprecation.showfileUploaderEncoding', False)
@st.cache(suppress_st_warning=True)
@st.cache(persist=True)

# helper function to predict images
def imagepreds(image):
    
    # load our serialized face detector model from disk
    prototxtPath = os.path.sep.join(['Model', "deploy.prototxt"])
    weightsPath = os.path.sep.join(['Model', "res10_300x300_ssd_iter_140000.caffemodel"])
    net = cv2.dnn.readNet(prototxtPath, weightsPath)

    # load the face mask detector model from disk
    model = load_model('blood_noblood_classifier.model')
        
    orig = image.copy()
    (h, w) = image.shape[:2]

    # construct a blob from the image
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the face detections
    net.setInput(blob)
    detections = net.forward()

    # loop over the detections
    for i in range(0, detections.shape[2]):
        
        # extract the confidence (i.e., probability) associated with
        # the detection
        confidence = detections[0, 0, i, 2]
        
        # filter out weak detections by ensuring the confidence is
        # greater than the minimum confidence
        
        if confidence > 0.6:
        
            # compute the (x, y)-coordinates of the bounding box for
            # the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            
            # ensure the bounding boxes fall within the dimensions of
            # the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # extract the face ROI, convert it from BGR to RGB channel
            # ordering, resize it to 224x224, and preprocess it
            face = image[startY:endY, startX:endX]
            #face = cv2.cvtColor(face, 1)
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)
            
            # pass the face through the model to determine if the face
            # has a mask or not
            (blood, noblood) = model.predict(face)[0]


            # determine the class label and color we'll use to draw
            # the bounding box and text
            label = "Blood" if blood > noblood else "No Blood"
            color = (0, 0, 255) if label == "Blood" else (0, 255, 0)
            
            # include the probability in the label
            label = "{}: {:.2f}%".format(label, max(blood, noblood) * 100)
            
            # display the label and bounding box rectangle on the output
            # frame
            cv2.putText(image, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
        
    # show the output image
    st.image(image, caption='Predictions', width=720)
    # cv2.imshow("Output", image)
    # cv2.waitKey(0)


# helper function to predict in real-time
def videopreds():
    # define a helper function to detected face and bounding box for each image 
    # in a live video frame
    def detect_and_predict_blood(frame, faceNet, bloodNet):
        
        # grab the dimensions of the frame and then construct a blob
        # from it
        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))
        
        # pass the blob through the network and obtain the face detections
        faceNet.setInput(blob)
        detections = faceNet.forward()
        
        # initialize our list of faces, their corresponding locations,
        # and the list of predictions from our face mask network
        faces = []
        locs = []
        preds = []

        # loop over the detections
        for i in range(0, detections.shape[2]):
            
            # extract the confidence (i.e., probability) associated with
            # the detection
            confidence = detections[0, 0, i, 2]
            
            # filter out weak detections by ensuring the confidence is
            # greater than the minimum confidence
            if confidence > 0.5:
                
                # compute the (x, y)-coordinates of the bounding box for
                # the object
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                
                # ensure the bounding boxes fall within the dimensions of
                # the frame
                (startX, startY) = (max(0, startX), max(0, startY))
                (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

                # extract the face ROI, convert it from BGR to RGB channel
                # ordering, resize it to 224x224, and preprocess it
                face = frame[startY:endY, startX:endX]
                face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                face = cv2.resize(face, (224, 224))
                face = img_to_array(face)
                face = preprocess_input(face)
                
                # add the face and bounding boxes to their respective
                # lists
                faces.append(face)
                locs.append((startX, startY, endX, endY))

                # only make a predictions if at least one face was detected
        if len(faces) > 0:
            
            # for faster inference we'll make batch predictions on *all*
            # faces at the same time rather than one-by-one predictions
            # in the above `for` loop
            faces = np.array(faces, dtype="float32")
            preds = bloodNet.predict(faces, batch_size=32)
            
        # return a 2-tuple of the face locations and their corresponding
        # locations
        return (locs, preds)

    # load our serialized face detector model from disk
    prototxtPath = os.path.sep.join(['Model', "deploy.prototxt"])
    weightsPath = os.path.sep.join(['Model', "res10_300x300_ssd_iter_140000.caffemodel"])
    faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

    # load the face mask detector model from disk
    bloodNet = load_model('blood_noblood_classifier.model')

    # initialize the video stream and allow the camera sensor to warm up
    # vs = VideoStream(src=0).start()
    
    # time.sleep(2.0)
    @st.cache(allow_output_mutation=True)
    def get_cap():
        return cv2.VideoCapture(0)

    cap = get_cap()

    frameST = st.empty()
    #param=st.sidebar.slider('chose your value')

    # loop over the frames from the video stream
    while True:
        
        # grab the frame from the threaded video stream and resize it
        # to have a maximum width of 400 pixels
        # frame = vs.read()
        # frame = imutils.resize(frame, width=400)
        ret, frame = cap.read()
        # detect faces in the frame and determine if they are wearing a
        # face mask or not
        (locs, preds) = detect_and_predict_blood(frame, faceNet, bloodNet)

        # loop over the detected face locations and their corresponding
        # locations
        for (box, pred) in zip(locs, preds):
            
            # unpack the bounding box and predictions
            (startX, startY, endX, endY) = box
            (blood, noblood) = pred
            
            # determine the class label and color we'll use to draw
            # the bounding box and text
            label = "Blood" if blood > noblood else "No Blood"
            color = (0, 0, 255) if label == "Blood" else (0, 255, 0)
            
            # include the probability in the label
            label = "{}: {:.2f}%".format(label, max(blood, noblood) * 100)
            
            # display the label and bounding box rectangle on the output
            # frame
            cv2.putText(frame, label, (startX, startY - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

        # show the output frame
        # cv2.imshow("Frame", frame)
        # key = cv2.waitKey(1) & 0xFF
        frameST.image(frame, channels="BGR")
        
        # if the `q` key was pressed, break from the loop
        # if key == ord("q"):
        #     break
            
    # do a bit of cleanup
    cv2.destroyAllWindows()
    #vs.stop()

@st.cache(persist=True)
def load_image(img):
    im = Image.open(img)
    return im


ig = Image.open('sample.jpg')
st.image(ig, width=920)

st.title('Blood Detection Classifier')

st.text('Build with Streamlit,Tensorflow, Keras and OpenCV By Mrinal Walia')

st.header("Select the options from sidebar: ")

st.subheader("Image Detection: For uploading an Image")

st.subheader("Video Detection: For opening the webcam and checking the results")

st.subheader("Performance Metrics: To check various performance metrices")

st.markdown("THANKS FOLKS!!")

st.subheader("Happy Learning")

st.subheader("Creator: @MRINAL WALIA")

st.markdown('Source Code Link: https://github.com/abhiwalia15/Face-Classification-into-Blood-No-Blood')

def main():

	menu = ['Image Detection', 'Video Detection', 'Perofrmance Metrics']
	choice = st.sidebar.selectbox('Menu',menu)

	if choice == 'Image Detection':
		st.subheader('**Blood Detection**')
		img_file_buffer = st.file_uploader("Upload an image")
		
		if img_file_buffer is not None:
			image = Image.open(img_file_buffer)
			img_array = np.array(image)
			imagepreds(img_array)

	elif choice == 'Video Detection':
		st.subheader('**Blood Detection**')
		videopreds()

	elif choice == 'Perofrmance Metrics':

		st.subheader('About Performance Metrics')

		st.info("CLASSIFICATION REPORT")
		cr = Image.open('Results/classification_report.png')
		st.image(cr, width=320)

		st.info("CONFUSION MATRIX")
		cm = Image.open('Results/confusion_matrix.png')
		st.image(cm, width=320)

		st.info("ACCURACY SCORE")
		acs = Image.open('Results/accuracy_score.png')
		st.image(acs, width=320)


# driver code
if __name__ == '__main__':
    main()

2021-10-25 18:41:53.293 
  command:

    streamlit run C:\Users\Ayushman\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [4]:
streamlit run C:\Users\Ayushman\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py

SyntaxError: invalid syntax (Temp/ipykernel_752/67222794.py, line 1)